# Dependencies

In [1]:
# Important
#import utils
import requests
import json
import numpy as np
import pandas as pd

# Helpers
from itertools import islice
from tqdm import tqdm
import time
from itertools import islice, product

# Miscellaneous, maybe not used
#from langdetect import detect, DetectorFactory
#DetectorFactory.seed=0

# Obtain data through a series of api requests

In [ ]:
import os
from urllib.parse import quote
from dotenv import load_dotenv
load_dotenv()

app_id = "281990" # Stellaris  #"221001" # FTL

# Setup requests
data_list = []
cursor = "*" # Start cursor is "*"
I_want_to_keep_going = True
n_tot_reviews = 0 # Count number of loaded reviews

while I_want_to_keep_going and n_tot_reviews < 1000000:
    url = f"https://store.steampowered.com/appreviews/{app_id}?json=1&filter=updated&cursor={quote(cursor)}&review_type=all&purchase_type=all&num_per_page=100&language=english&day_range=365?"#key={os.environ["STEAM"]}"
    #url = "https://store.steampowered.com/appreviews/10" + cursor + "?json=1&filter=updated"
    response = requests.get(url)
    data = json.loads(response.content)

    # Get a query summary from first request
    if cursor == "*":
        print(data["query_summary"])
    
    # Print the success code ("1" is good)
    #print(data["success"])

    # Keep track of number of reviews
    n_tot_reviews += len(data["reviews"])

    # Handle potential issues
    try:
        if cursor == data["cursor"]: I_want_to_keep_going = False
        else:
            try:
                cursor = data["cursor"]
                data_list.append(data)
            except:
                break
    except:
        break

#params["cursor"] = data1["cursor"]

# cursor1 = data1["cursor"]
# response2 = requests.get("https://store.steampowered.com/appreviews/" + app_id + "&cursor=" + cursor1, params=params)
# data2 = json.loads(response2.content)
# print(data2["cursor"])


#data["reviews"][12]["review"] #12 is rev index

print("\nObtained " + str(sum([len(i["reviews"]) for i in data_list])) + " reviews.")
#print("\nLengths:")
#[print(len(i["reviews"])) for i in data_list]

#print("\nCursors:")
#[print(i["cursor"]) for i in data_list]#, data1["cursor"], " and Cursor 2: ", data2["cursor"], " Equal: ", data1["cursor"] == data2["cursor"])

#print("\nFirst reviews:")
#[print(i["reviews"][0]["review"] + "\n-----") for i in data_list] # data1["reviews"][0]["review"] == data2["reviews"][0]["review"])

{'num_reviews': 100, 'review_score': 8, 'review_score_desc': 'Very Positive', 'total_positive': 83709, 'total_negative': 11265, 'total_reviews': 94974}


# Format the data into a dataframe

In [14]:
# Each element in data_list is a "package" of a couple of reviews.

# Maka a list of tuples representing our observations
data = []

for pack in data_list:
    for review in pack["reviews"]:
        # If the dev has responded, two additional "columns"/keys are included. We skip them because they are very few.
        if 'timestamp_dev_responded' in review.keys() and 'developer_response' in review.keys():
            review.pop('timestamp_dev_responded')
            review.pop('developer_response')
        data.append(review.values())

# Make a list of the keys present for each individual review
colnames = list(data_list[1]["reviews"][0].keys())

df = pd.DataFrame(data, columns=colnames)
df

,recommendationid,author,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,primarily_steam_deck
0,184668559,"{'steamid': '76561197986126875', 'num_games_ow...",english,"Great game, but the end-game is not very inter...",1735931110,1736153281,True,0,1,0,0,True,False,False,False
1,184908323,"{'steamid': '76561198159203912', 'num_games_ow...",english,fun if you have too much free time,1736151911,1736151911,True,0,0,0,0,True,False,False,False
2,168118609,"{'steamid': '76561199132338764', 'num_games_ow...",english,> `\n> invincible\n> instant_build\n> max_reso...,1719260702,1736147489,True,0,0,0,0,False,False,False,False
3,184902224,"{'steamid': '76561198431566992', 'num_games_ow...",english,I know you may be tempted to buy this but in ...,1736142633,1736142633,False,2,0,0.547325074672698975,0,True,False,False,False
4,184901085,"{'steamid': '76561198118774524', 'num_games_ow...",english,great,1736141064,1736141064,True,0,0,0,0,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94942,22881655,"{'steamid': '76561198008837363', 'num_games_ow...",english,The menu looks nice,1462810353,1462810353,True,16,16,0.388136118650436401,1,True,False,False,False
94943,22881630,"{'steamid': '76561198090386692', 'num_games_ow...",english,Game of thrones in space!!!,1462810229,1462810229,True,5,3,0.295664846897125244,0,True,False,False,False
94944,22881480,"{'steamid': '76561198005610307', 'num_games_ow...",english,https://www.youtube.com/watch?v=hvmcvfzHEjY\n\...,1462809610,1462809943,True,482,25,0.718119204044342041,30,False,True,False,False
94945,22881467,"{'steamid': '76561198005120097', 'num_games_ow...",english,[h1]FULL VIDEO REVIEW:[/h1] https://youtu.be/i...,1462809557,1462809605,True,184,9,0.652377605438232422,15,False,True,False,False


# Filter the data

In [15]:
# Fix the funny votes (some have a huge score, so we set them to 5000, which is slightly higher than the max of the others.
df["votes_funny"] = [i if i < 5000 else 5000 for i in df["votes_funny"]]

# Filter out reviews that occured less than [one month ago] #maybe change this frame?
df = df[time.time() - df["timestamp_created"] > 30*24*3600]

# Save the data locally

In [16]:
df.to_csv("data.csv") # Overwrites